In [8]:
import os
import glob
import os.path
import re
import pickle
import datetime


import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

import matplotlib.pyplot as plt

+ 强制全部输出间隔一致

In [23]:
class AtChem2Case:
    def __init__(self, mDir, facfile = None):
        assert os.path.exists(mDir)
        self.modelpath = mDir
        
        self.facfile = facfile
        if not facfile:
            self.facfile = glob.glob(f"{mDir}/*.fac")[0]
        
        self.mechDG = self._init_DG_from_fac(self.facfile)
        self.frames = {}


        self.resolve_configuration()
        if os.path.exists(f"{mDir}/output"):
            self.resolve_output(f"{mDir}/output")

    def _init_DG_from_fac(self, facfile: str):
        assert os.path.exists(facfile)

        lines = open(facfile).read().splitlines()
        lines_reaction = [L for L in lines if L.startswith("% ")]
        reactions = list(map(lambda x: re.search(': (.*) ;', x).group(1).replace(' ', ''), lines_reaction))
    
        for iL, L in enumerate(lines):
            if L == 'VARIABLE':
                break
        iL += 1
        variables = []
        while not lines[iL].startswith('******************'):
            variables.extend([ele for ele in re.split(r'[; ]', lines[iL]) if ele.strip()])
            iL += 1

        
        DG = nx.DiGraph()
        for sp in variables:
            DG.add_node(sp, type="species")
        for reac in reactions:
            DG.add_node(reac, type="reaction")
            reactants = re.findall(r"\w+", reac.split('=')[0])
            products = re.findall(r"\w+", reac.split('=')[1])
            for rtt in reactants:
                DG.add_edge(rtt, reac)
            for pdt in products:
                DG.add_edge(reac, pdt)

        return DG    

    def resolve_configuration(self):
        for k, v in self.get_model_parameters().items():
            setattr(self, k, v)
        self.timezone = -float(self.lon) / 15
        self.startDatetime = datetime.datetime.strptime(f"""{self.year}/{self.month}/{self.day}""", "%Y/%m/%d")
        self.startDatetime += datetime.timedelta(hours=self.timezone)
        self.startDatetime += datetime.timedelta(seconds=int(self.starttime))
        self.timeseries = [self.startDatetime]
        timestep = datetime.timedelta(seconds=int(self.dt))
        for i in range(int(self.nsteps)):
            self.timeseries.append(self.timeseries[-1] + timestep)
        
    def get_model_parameters(self):
        configfile = os.path.join(self.modelpath, "configuration/model.parameters")
        model_params_lines = [L.split()[0] for L in open(configfile, "r").read().splitlines()]
        keys_by_order = ("nsteps", "dt", "species_interp_method", "condition_interp_method", "dt_out", "starttime", "dt_jacobian_out", 
                         "lat", "lon", "day", "month", "year", "dt_rrate_out")
        model_parameters = {keys_by_order[i]:model_params_lines[i] for i in range(len(model_params_lines))}
        return model_parameters
    
    def resolve_output(self, oDir: str):
        # spe_conc_file = f"{self.modelpath}/output/speciesConcentrations.output"
        pass

    def _get_output_times(self):
        dfT = pd.read_csv(f'{self.modelpath}/speciesConcentrations.output', sep=r'\s+', index_col=0)
        outdts = []
        for t in dfT.index:
            outdts.append(startDatetime + datetime.timedelta(seconds=int(t)))
        return outdts
    
    def __getitem__(self, time: int):
        return self.frames[time]

In [20]:
rct = AtChem2Case("/home/roadelse/Model/AtChem2/AtChem2-1.2.2/model")

In [21]:
rct.startDatetime

datetime.datetime(2019, 6, 21, 8, 0)

In [22]:
rct.timeseries

[datetime.datetime(2019, 6, 21, 8, 0),
 datetime.datetime(2019, 6, 21, 8, 5),
 datetime.datetime(2019, 6, 21, 8, 10),
 datetime.datetime(2019, 6, 21, 8, 15),
 datetime.datetime(2019, 6, 21, 8, 20),
 datetime.datetime(2019, 6, 21, 8, 25),
 datetime.datetime(2019, 6, 21, 8, 30),
 datetime.datetime(2019, 6, 21, 8, 35),
 datetime.datetime(2019, 6, 21, 8, 40),
 datetime.datetime(2019, 6, 21, 8, 45),
 datetime.datetime(2019, 6, 21, 8, 50),
 datetime.datetime(2019, 6, 21, 8, 55),
 datetime.datetime(2019, 6, 21, 9, 0),
 datetime.datetime(2019, 6, 21, 9, 5),
 datetime.datetime(2019, 6, 21, 9, 10),
 datetime.datetime(2019, 6, 21, 9, 15),
 datetime.datetime(2019, 6, 21, 9, 20),
 datetime.datetime(2019, 6, 21, 9, 25),
 datetime.datetime(2019, 6, 21, 9, 30),
 datetime.datetime(2019, 6, 21, 9, 35),
 datetime.datetime(2019, 6, 21, 9, 40),
 datetime.datetime(2019, 6, 21, 9, 45),
 datetime.datetime(2019, 6, 21, 9, 50),
 datetime.datetime(2019, 6, 21, 9, 55),
 datetime.datetime(2019, 6, 21, 10, 0),
 dat

In [4]:
spe_conc_file = f"{rct.modelpath}/output/speciesConcentrations.output"
env_file = f"{rct.modelpath}/output/environmentVariables.output"
rates_dir = f"{rct.modelpath}/output/reactionRates"